<a href="https://colab.research.google.com/github/unerue/computer-vision/blob/master/lenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip uninstall -y torchsummary
!pip install torch-summary

In [3]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms
from torchsummary import summary

In [4]:
class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val: float, n: int = 1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [5]:
transform = transforms.Compose([
    transforms.ToTensor()
])

train_set = torchvision.datasets.MNIST(
    './mnist', train=True, download=True, transform=transform)
valid_set = torchvision.datasets.MNIST(
    './mnist', train=False, download=False, transform=transform)

Extracting ./mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist/MNIST/raw


Extracting ./mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist/MNIST/raw


Extracting ./mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist/MNIST/raw


Extracting ./mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist/MNIST/raw
Processing...
Done!


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [17]:
print(train_set.data.shape)
print(valid_set.data.shape)

train_loader = DataLoader(train_set, batch_size=128, num_workers=2, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=512, num_workers=2, shuffle=False)

torch.Size([60000, 28, 28])
torch.Size([10000, 28, 28])


In [12]:
class LeNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(
            in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=0)
        self.conv2 = nn.Conv2d(
            in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.fc1 = nn.Linear(4*4*16, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = F.max_pool2d(out, 2)
        out = F.relu(self.conv2(out))
        out = F.max_pool2d(out, 2)
        out = out.view(out.size(0), -1)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out


device = 'cuda' if torch.cuda.is_available() else 'cpu'
net = LeNet().to(device)
print(summary(net, input_data=(1, 28, 28), verbose=0))

------------------------------------------------------------------------------------------
Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 6, 24, 24]           156
├─Conv2d: 1-2                            [-1, 16, 8, 8]            2,416
├─Linear: 1-3                            [-1, 120]                 30,840
├─Linear: 1-4                            [-1, 84]                  10,164
├─Linear: 1-5                            [-1, 10]                  850
Total params: 44,426
Trainable params: 44,426
Non-trainable params: 0
Total mult-adds (M): 0.28
------------------------------------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.04
Params size (MB): 0.17
Estimated Total Size (MB): 0.21
------------------------------------------------------------------------------------------


In [18]:
optimizer = optim.SGD(net.parameters(), lr=1e-3, momentum=0.9)
loss_fn = nn.CrossEntropyLoss()

In [19]:
def train_fn(epoch, model, data_loader, loss_fn, optimizer):
    model.train()
    correct = 0
    total = len(data_loader.dataset)
    batch = 0
    losses = AverageMeter()
    corrects = AverageMeter()
    
    for i, (inputs, labels) in enumerate(data_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        
        optimizer.zero_grad()

        outputs = model(inputs)
        loss = loss_fn(outputs, labels)

        loss.backward()
        optimizer.step()
        
        batch += inputs.size(0)

        _, preds = torch.max(outputs, 1)
        
        correct += (preds == labels).sum().item()

        if (i+1) % 50 == 0:
            print(f'Epoch {epoch+1}({(batch/total)*100:.2f}%) Train loss: {loss.item():.4f} accuracy: {correct/batch:.4f}')
        
        losses.update(loss, inputs.size(0))
        corrects.update(correct, inputs.size(0))
        
    return losses.avg, corrects.avg

In [20]:
epochs = 3

train_losses = []
valid_losses = []

train_accuracies = []
valid_accuracies = []

for epoch in range(epochs):
    loss, acc = train_fn(epoch, net, train_loader, loss_fn, optimizer)
    train_losses.append(loss)
    train_accuracies.append(acc)

Epoch 1(10.67%) Train loss: 2.3000 accuracy: 0.0923
Epoch 1(21.33%) Train loss: 2.2983 accuracy: 0.0970
Epoch 1(32.00%) Train loss: 2.3009 accuracy: 0.0948
Epoch 1(42.67%) Train loss: 2.3060 accuracy: 0.0956
Epoch 1(53.33%) Train loss: 2.2966 accuracy: 0.0972
Epoch 1(64.00%) Train loss: 2.2987 accuracy: 0.0991
Epoch 1(74.67%) Train loss: 2.2985 accuracy: 0.0982
Epoch 1(85.33%) Train loss: 2.2984 accuracy: 0.0982
Epoch 1(96.00%) Train loss: 2.2913 accuracy: 0.0988
Epoch 2(10.67%) Train loss: 2.3002 accuracy: 0.1152
Epoch 2(21.33%) Train loss: 2.2877 accuracy: 0.1601
Epoch 2(32.00%) Train loss: 2.2896 accuracy: 0.1787
Epoch 2(42.67%) Train loss: 2.2900 accuracy: 0.1914
Epoch 2(53.33%) Train loss: 2.2851 accuracy: 0.1990
Epoch 2(64.00%) Train loss: 2.2839 accuracy: 0.2089
Epoch 2(74.67%) Train loss: 2.2716 accuracy: 0.2199
Epoch 2(85.33%) Train loss: 2.2726 accuracy: 0.2318
Epoch 2(96.00%) Train loss: 2.2542 accuracy: 0.2442
Epoch 3(10.67%) Train loss: 2.2267 accuracy: 0.3698
Epoch 3(21.3